### Projeto SQL 
* Este projeto tem a finalidade de analisar dados sobre livros, autores, avaliações, com a finalidade de gerar uma proposição válida para o novo produto em desenvolvimento. 
* Serão executadas as seguintes tarefas: 
* Encontrar o número de livros lançados depois de 1 de janeiro de 2000.
* Encontrar o número de avaliações e a classificação média para cada livro.
* Identificar a editora que lançou o maior número de livros com mais de 50 páginas (isso vai ajudar você a excluir brochuras e publicações parecidas da sua análise).
* Identificar o autor com a média mais alta classificação de livros: olhe apenas para livros com pelo menos 50 classificações.
* Encontrar o número médio de avaliações entre usuários que classificaram mais do que 50 livros.


In [2]:
#importar bibliotecas
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'practicum_student', # nome de usuário
'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # senha
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # porta de conexão
'db': 'data-analyst-final-project-db'} # o nome do banco de dados
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# Consulta SQL no data frame books (livros) e selecionando as 5 primeiras linhas
query = "SELECT * FROM books LIMIT 5"
pd.io.sql.read_sql(query, con=engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# Consulta SQL no data frame authors (autores) e selecionando as 5 primeiras linhas
query = "SELECT * FROM authors LIMIT 5"
pd.io.sql.read_sql(query, con=engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# Consulta SQL no data frame publishers (editoras) e selecionando as 5 primeiras linhas
query = "SELECT * FROM publishers LIMIT 5"
pd.io.sql.read_sql(query, con=engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
# Consulta SQL no data frame ratings (classificações) e selecionando as 5 primeiras linhas
query = "SELECT * FROM ratings LIMIT 5"
pd.io.sql.read_sql(query, con=engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
# Consulta SQL no data frame reviews (avaliações)  e selecionando as 5 primeiras linhas
query = "SELECT * FROM reviews  LIMIT 5"
pd.io.sql.read_sql(query, con=engine)


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [8]:
# Consulta SQL para contar o número de livros lançados após 1 de janeiro de 2000
query = "SELECT COUNT(*) as num_books FROM books WHERE publication_date > '2000-01-01'"

# Executando a consulta e armazena o resultado em um DataFrame
result = pd.io.sql.read_sql(query, con=engine)

num_books = result.iloc[0]['num_books']
print(f" O número de livros lançados depois de 1 de janeiro de 2000 é {num_books}")


 O número de livros lançados depois de 1 de janeiro de 2000 é 819


In [17]:
# Encontrando o número de avaliações e a classificação média para cada livro.
query = """
SELECT b.book_id, 
       b.author_id, 
       b.title, 
       b.num_pages, 
       b.publication_date, 
       b.publisher_id,
       COALESCE(rv.num_reviews, 0) as num_reviews, 
       COALESCE(rt.avg_rating, 0) as avg_rating
FROM books b
LEFT JOIN (
    SELECT book_id, COUNT(DISTINCT review_id) as num_reviews
    FROM reviews
    GROUP BY book_id
) rv ON b.book_id = rv.book_id
LEFT JOIN (
    SELECT book_id, AVG(rating) as avg_rating
    FROM ratings
    GROUP BY book_id
) rt ON b.book_id = rt.book_id
ORDER BY num_reviews DESC
"""

books_df = pd.read_sql(query, con=engine)

books_df.head()


,book_id,author_id,title,num_pages,publication_date,publisher_id,num_reviews,avg_rating
0,948,554,Twilight (Twilight #1),501,2006-09-06,176,7,3.662500
1,734,278,The Glass Castle,288,2006-01-17,258,6,4.206897
2,695,395,The Curious Incident of the Dog in the Night-Time,226,2004-05-18,309,6,4.081081
3,854,99,The Road,241,2006-09-26,9,6,3.772727
4,733,372,The Giver (The Giver #1),208,2006-01-24,101,6,3.750000


* O data frame acima contêm os nomes dos livros, quantas avaliações eles tiveram e a classificação média para cada livro. 
* O data frame está ordenado pela quantidade de reviews em ordem decrescente.

In [11]:
# Identificando a editora que lançou o maior número de livros com mais de 50 páginas
query = """
SELECT p.publisher_id, p.publisher, COUNT(b.book_id) as num_books
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY num_books DESC
LIMIT 1
"""
pd.read_sql(query, con=engine)



,publisher_id,publisher,num_books
0,212,Penguin Books,42


* A editora que lançou o maior número de livros com mais de 50 páginas foi a Penguin Books.	

In [12]:
# Identificando o autor com a média mais alta na classificação de livros em livros com pelo menos 50 classificações.
query = """
WITH filtered_books AS (
    SELECT r.book_id, b.author_id, AVG(r.rating) as avg_rating, COUNT(r.rating_id) as num_ratings
    FROM ratings r
    JOIN books b ON r.book_id = b.book_id
    GROUP BY r.book_id, b.author_id
    HAVING COUNT(r.rating_id) >= 50
),
author_ratings AS (
    SELECT fb.author_id, AVG(fb.avg_rating) as author_avg_rating
    FROM filtered_books fb
    GROUP BY fb.author_id
)
SELECT a.author_id, a.author, ar.author_avg_rating
FROM authors a
JOIN author_ratings ar ON a.author_id = ar.author_id
ORDER BY ar.author_avg_rating DESC
LIMIT 1
"""
pd.read_sql(query, con=engine)


,author_id,author,author_avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844


In [13]:
# Encontrando o número médio de avaliações entre usuários que classificaram mais do que 50 livros.
query = """
WITH users_with_more_than_50_ratings AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 50
)
SELECT AVG(user_reviews.num_reviews) as avg_reviews
FROM (
    SELECT rv.username, COUNT(rv.review_id) as num_reviews
    FROM users_with_more_than_50_ratings u
    JOIN reviews rv ON u.username = rv.username
    GROUP BY rv.username
) user_reviews
"""

result = pd.read_sql(query, con=engine)

avg_reviews = result.iloc[0, 0]
print(f"O número médio de avaliações entre usuários que classificaram mais do que 50 livros é {avg_reviews:.2f}")


O número médio de avaliações entre usuários que classificaram mais do que 50 livros é 24.33
